In [38]:
# Us U.S. Census API Key to call housing related data (median home value, state, county, )
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import gmaps

# Census & gmaps API Keys
from config import (census_api_key)
c = Census(census_api_key, year=2018)

import us
us.states.IN
us.states.IN.fips
us.states.IN.name
us.states.IN.is_contiguous
True

True

Link to U.S. Census 2018 ACS variables:
https://api.census.gov/data/2018/acs/acs5/variables.html
also helpful:
https://www.census.gov/data/developers/data-sets/acs-5year.html

In [39]:
# Run Census Search to retrieve data on all states
us_census_data = c.acs5.get(("NAME",
                          "B25077_001E",
                          "B19013_001E",
                          "B01003_001E",
                          "B01002_001E",
                          "B19301_001E",),
                         {'for': 'county:*'})

# Convert to DataFrame
us_census_df = pd.DataFrame(us_census_data)

"""
us_census_pd = pd.DataFrame(us_census_data,
                            dtype={'Population': 'int',
                                  'Median Age': 'int',                               
                            },
                            converters={'Median Home Value': convert_currency,
                                        'Household Income': convert_currency,
                                        'Per Capita Income': convert_currency}
                          )
""" 


us_census_df = us_census_df.rename(columns=
                             {"B25077_001E": "Median Home Value",
                              "B19013_001E": "Household Income",
                              "B01003_001E": "Population",
                              "B01002_001E": "Median Age",
                              "B19301_001E": "Per Capita Income",
                              "state": "State",
                              "county": "County"
                             })


us_census_df.head(1)

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
0,"Washington County, Mississippi",74700.0,30834.0,47086.0,36.9,19884.0,28,151


In [40]:
us_census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 8 columns):
NAME                 3220 non-null object
Median Home Value    3220 non-null float64
Household Income     3219 non-null float64
Population           3220 non-null float64
Median Age           3220 non-null float64
Per Capita Income    3219 non-null float64
State                3220 non-null object
County               3220 non-null object
dtypes: float64(5), object(3)
memory usage: 201.4+ KB


In [41]:
# Convert float64 to int
"""
us_census_df['Median Age'].astype('int')
us_census_df['Median Home Value'].astype('int')
"""

"\nus_census_df['Median Age'].astype('int')\nus_census_df['Median Home Value'].astype('int')\n"

In [42]:
# Convert float64 to int
"""
us_census_df['Median Age'] = us_census_df['Median Age'].astype('int')
us_census_df['Median Home Value'] = us_census_df['Median Home Value'].astype('int')

us_census_df.dtypes
"""

"\nus_census_df['Median Age'] = us_census_df['Median Age'].astype('int')\nus_census_df['Median Home Value'] = us_census_df['Median Home Value'].astype('int')\n\nus_census_df.dtypes\n"

In [43]:
us_census_df.sort_values(['State', 'County'], ascending=[True, True])

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
2517,"Autauga County, Alabama",147900.0,58786.0,55200.0,37.8,29372.0,01,001
2502,"Baldwin County, Alabama",189800.0,55962.0,208107.0,42.8,31203.0,01,003
985,"Barbour County, Alabama",92900.0,34186.0,25782.0,39.9,18461.0,01,005
2513,"Bibb County, Alabama",96500.0,45340.0,22527.0,39.9,20199.0,01,007
2516,"Blount County, Alabama",124700.0,48695.0,57645.0,40.8,22656.0,01,009
...,...,...,...,...,...,...,...,...
2492,"Vega Baja Municipio, Puerto Rico",107600.0,19096.0,53371.0,40.7,10601.0,72,145
919,"Vieques Municipio, Puerto Rico",104200.0,15539.0,8771.0,43.6,11156.0,72,147
2480,"Villalba Municipio, Puerto Rico",90900.0,19855.0,22993.0,38.8,10677.0,72,149
920,"Yabucoa Municipio, Puerto Rico",88100.0,16013.0,34149.0,42.5,9148.0,72,151


In [44]:
county = us_census_df.NAME.str.split(', ', expand=True).loc[:,0]
state = us_census_df.NAME.str.split(', ', expand=True).loc[:,1]

In [45]:
us_census_df["County"]=county
us_census_df
us_census_df["State"]=state
us_census_df.head()

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
0,"Washington County, Mississippi",74700.0,30834.0,47086.0,36.9,19884.0,Mississippi,Washington County
1,"Perry County, Mississippi",83700.0,39007.0,12028.0,40.9,21611.0,Mississippi,Perry County
2,"Choctaw County, Mississippi",81600.0,37203.0,8321.0,44.0,20589.0,Mississippi,Choctaw County
3,"Itawamba County, Mississippi",89500.0,40510.0,23480.0,40.0,20629.0,Mississippi,Itawamba County
4,"Carroll County, Mississippi",89300.0,43060.0,10129.0,47.0,22567.0,Mississippi,Carroll County


In [46]:
us_census_df.sort_values(['State', 'County'], ascending=[True, True])

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
2517,"Autauga County, Alabama",147900.0,58786.0,55200.0,37.8,29372.0,Alabama,Autauga County
2502,"Baldwin County, Alabama",189800.0,55962.0,208107.0,42.8,31203.0,Alabama,Baldwin County
985,"Barbour County, Alabama",92900.0,34186.0,25782.0,39.9,18461.0,Alabama,Barbour County
2513,"Bibb County, Alabama",96500.0,45340.0,22527.0,39.9,20199.0,Alabama,Bibb County
2516,"Blount County, Alabama",124700.0,48695.0,57645.0,40.8,22656.0,Alabama,Blount County
...,...,...,...,...,...,...,...,...
881,"Sweetwater County, Wyoming",205800.0,73008.0,44117.0,34.6,32624.0,Wyoming,Sweetwater County
880,"Teton County, Wyoming",827400.0,83831.0,23059.0,39.3,53703.0,Wyoming,Teton County
890,"Uinta County, Wyoming",177500.0,58235.0,20609.0,35.5,27009.0,Wyoming,Uinta County
891,"Washakie County, Wyoming",162700.0,53426.0,8129.0,43.5,27556.0,Wyoming,Washakie County


In [47]:
us_census_df.to_csv("us_census_file.csv")

In [48]:
IN_census=us_census_df.loc[us_census_df["State"]=="Indiana"]
IN_census

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
2220,"Monroe County, Indiana",167900.0,47075.0,145403.0,29.3,27461.0,Indiana,Monroe County
2221,"Fountain County, Indiana",94900.0,49887.0,16486.0,43.1,26721.0,Indiana,Fountain County
2222,"Elkhart County, Indiana",137900.0,55399.0,203604.0,35.5,25075.0,Indiana,Elkhart County
2223,"Crawford County, Indiana",89700.0,43875.0,10581.0,43.6,21021.0,Indiana,Crawford County
2224,"Cass County, Indiana",85500.0,47545.0,38084.0,40.8,23918.0,Indiana,Cass County
...,...,...,...,...,...,...,...,...
3189,"Carroll County, Indiana",131600.0,56429.0,19994.0,42.8,28430.0,Indiana,Carroll County
3190,"Huntington County, Indiana",105000.0,52208.0,36378.0,40.7,25386.0,Indiana,Huntington County
3191,"White County, Indiana",110300.0,55086.0,24217.0,41.9,27621.0,Indiana,White County
3192,"Jay County, Indiana",85400.0,44772.0,20993.0,40.0,22470.0,Indiana,Jay County


In [49]:
IN_census.sort_values(['State', 'County'], ascending=[True, True])

,NAME,Median Home Value,Household Income,Population,Median Age,Per Capita Income,State,County
2284,"Adams County, Indiana",122400.0,51713.0,35195.0,33.4,22514.0,Indiana,Adams County
2247,"Allen County, Indiana",123800.0,53402.0,370016.0,35.8,28239.0,Indiana,Allen County
2277,"Bartholomew County, Indiana",143800.0,59045.0,81893.0,37.7,30515.0,Indiana,Bartholomew County
3175,"Benton County, Indiana",87900.0,46188.0,8667.0,40.5,23322.0,Indiana,Benton County
2258,"Blackford County, Indiana",71300.0,42217.0,12129.0,44.3,24663.0,Indiana,Blackford County
...,...,...,...,...,...,...,...,...
3177,"Washington County, Indiana",113200.0,47078.0,27827.0,41.5,24048.0,Indiana,Washington County
3170,"Wayne County, Indiana",97400.0,44228.0,66613.0,40.9,25460.0,Indiana,Wayne County
3183,"Wells County, Indiana",120700.0,56265.0,27947.0,39.8,27106.0,Indiana,Wells County
3191,"White County, Indiana",110300.0,55086.0,24217.0,41.9,27621.0,Indiana,White County


In [50]:
IN_census.count()

NAME                 92
Median Home Value    92
Household Income     92
Population           92
Median Age           92
Per Capita Income    92
State                92
County               92
dtype: int64

In [51]:
IN_census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 2220 to 3193
Data columns (total 8 columns):
NAME                 92 non-null object
Median Home Value    92 non-null float64
Household Income     92 non-null float64
Population           92 non-null float64
Median Age           92 non-null float64
Per Capita Income    92 non-null float64
State                92 non-null object
County               92 non-null object
dtypes: float64(5), object(3)
memory usage: 6.5+ KB


In [52]:
IN_census.describe()

,Median Home Value,Household Income,Population,Median Age,Per Capita Income
count,92.000000,92.000000,92.000000,92.000000,92.000000
mean,122713.043478,53914.510870,72145.934783,40.247826,26846.782609
std,31798.604632,8667.058077,121530.475098,3.332474,4124.233771
min,71300.000000,42217.000000,5887.000000,28.000000,21021.000000
25%,97350.000000,48233.000000,20856.500000,38.700000,24249.000000
50%,117400.000000,52150.000000,34061.500000,40.700000,25962.500000
75%,141625.000000,57031.500000,67639.750000,42.225000,28429.250000
max,249400.000000,94644.000000,944523.000000,49.200000,46597.000000


In [53]:
IN_census_1 = IN_census[['County','State','Median Home Value','Household Income','Per Capita Income','Population','Median Age' ]]
IN_census_1.head(1)

,County,State,Median Home Value,Household Income,Per Capita Income,Population,Median Age
2220,Monroe County,Indiana,167900.0,47075.0,27461.0,145403.0,29.3


In [54]:
IN_census_2 = IN_census_1.sort_values(['State', 'County'], ascending=[True, True])

In [55]:
IN_census_2.to_csv("IN_census_file.csv")

In [56]:
IN_census_2

,County,State,Median Home Value,Household Income,Per Capita Income,Population,Median Age
2284,Adams County,Indiana,122400.0,51713.0,22514.0,35195.0,33.4
2247,Allen County,Indiana,123800.0,53402.0,28239.0,370016.0,35.8
2277,Bartholomew County,Indiana,143800.0,59045.0,30515.0,81893.0,37.7
3175,Benton County,Indiana,87900.0,46188.0,23322.0,8667.0,40.5
2258,Blackford County,Indiana,71300.0,42217.0,24663.0,12129.0,44.3
...,...,...,...,...,...,...,...
3177,Washington County,Indiana,113200.0,47078.0,24048.0,27827.0,41.5
3170,Wayne County,Indiana,97400.0,44228.0,25460.0,66613.0,40.9
3183,Wells County,Indiana,120700.0,56265.0,27106.0,27947.0,39.8
3191,White County,Indiana,110300.0,55086.0,27621.0,24217.0,41.9


In [57]:
IN_census_3 = IN_census_2.style.format({
    'Median Home Value': '${0:,.0f}'.format, 
    'Household Income': '${0:,.0f}'.format,
    'Per Capita Income': '${:,.0f}'.format,
    'Population': '{0:,.0f}'.format,
    'Median Age': '{0:,.2f}'.format
    })

In [58]:
IN_census_3

,County,State,Median Home Value,Household Income,Per Capita Income,Population,Median Age
2284,Adams County,Indiana,"$122,400","$51,713","$22,514","35,195",33.40
2247,Allen County,Indiana,"$123,800","$53,402","$28,239","370,016",35.80
2277,Bartholomew County,Indiana,"$143,800","$59,045","$30,515","81,893",37.70
3175,Benton County,Indiana,"$87,900","$46,188","$23,322","8,667",40.50
2258,Blackford County,Indiana,"$71,300","$42,217","$24,663","12,129",44.30
2279,Boone County,Indiana,"$222,500","$80,567","$44,712","64,321",38.00
2235,Brown County,Indiana,"$186,000","$61,455","$33,617","15,034",49.20
3189,Carroll County,Indiana,"$131,600","$56,429","$28,430","19,994",42.80
2224,Cass County,Indiana,"$85,500","$47,545","$23,918","38,084",40.80
3193,Clark County,Indiana,"$137,800","$54,240","$28,525","115,702",38.90
